In [1]:
import numpy as np
from PIL import Image

class Sphere:
    def __init__(self, center, radius, color):
        self.center = np.array(center)
        self.radius = radius
        self.color = np.array(color)

    def ray_intersection(self, ray_origin, ray_direction):
        oc = ray_origin - self.center
        a = np.dot(ray_direction, ray_direction)
        b = 2 * np.dot(oc, ray_direction)
        c = np.dot(oc, oc) - self.radius ** 2

        discriminant = b ** 2 - 4 * a * c

        if discriminant < 0:
            return np.inf

        t1 = (-b + np.sqrt(discriminant)) / (2 * a)
        t2 = (-b - np.sqrt(discriminant)) / (2 * a)

        if t1 >= 0 and t2 >= 0:
            return min(t1, t2)
        elif t1 >= 0:
            return t1
        elif t2 >= 0:
            return t2
        else:
            return np.inf


class Camera:
    def __init__(self, position, target, orientation, aperture):
        self.position = np.array(position)
        self.target = np.array(target)
        self.orientation = np.array(orientation)
        self.aperture = aperture

        self.forward = self.target - self.position
        self.forward = self.forward / np.linalg.norm(self.forward)

        right = np.cross(self.forward, self.orientation)
        self.right = right / np.linalg.norm(right)

        self.up = np.cross(self.right, self.forward)

    def generate_ray_direction(self, x, y, width, height):
        aspect_ratio = width / height
        fov_tan = np.tan(self.aperture / 2)
        normalized_x = (2 * (x + 0.5) / width - 1) * fov_tan * aspect_ratio
        normalized_y = (1 - 2 * (y + 0.5) / height) * fov_tan

        direction = self.forward + self.right * normalized_x + self.up * normalized_y
        return direction / np.linalg.norm(direction)


class Light:
    def __init__(self, position, color):
        self.position = np.array(position)
        self.color = np.array(color)


def trace_ray(ray_origin, ray_direction, objects, lights):
    closest_t = np.inf
    closest_object = None

    for obj in objects:
        t = obj.ray_intersection(ray_origin, ray_direction)
        if t < closest_t:
            closest_t = t
            closest_object = obj

    if closest_object is None:
        return np.array([0, 0, 0])

    hit_point = ray_origin + ray_direction * closest_t
    normal = (hit_point - closest_object.center) / closest_object.radius

    # Ambient color
    ambient_color = closest_object.color * 0.1

    # Diffuse color
    diffuse_color = np.array([0, 0, 0])
    for light in lights:
        light_direction = light.position - hit_point
        light_distance = np.linalg.norm(light_direction)
        light_direction = light_direction / light_distance

        # Shadow check
        shadow_origin = hit_point + light_direction * 0.001
        shadow_t = np.inf
        for obj in objects:
            t = obj.ray_intersection(shadow_origin, light_direction)
            if 0.001 < t < light_distance:
                shadow_t = t
                break

        if shadow_t == np.inf:
            diffuse_intensity = np.maximum(np.dot(normal, light_direction), 0)
            diffuse_effect = closest_object.color * light.color
            diffuse_effect[0] *= diffuse_intensity
            diffuse_effect[1] *= diffuse_intensity
            diffuse_effect[2] *= diffuse_intensity
            
            diffuse_color += diffuse_effect

    color = ambient_color + diffuse_color
    return np.minimum(color, 255)


# Scene configuration
width = 800
height = 600

# Create spheres
sphere1 = Sphere(center=[0, 0, 0], radius=1, color=[255, 0, 0])
sphere2 = Sphere(center=[-1.5, 0, 2], radius=0.5, color=[0, 255, 0])
sphere3 = Sphere(center=[1.5, 0, 2], radius=0.5, color=[0, 0, 255])
objects = [sphere1, sphere2, sphere3]

# Create lights
light1 = Light(position=[-2, 2, -1], color=[255, 255, 255])
light2 = Light(position=[2, 2, -1], color=[255, 255, 255])
lights = [light1, light2]

# Camera configuration
camera = Camera(position=[0, 0, -5], target=[0, 0, 1], orientation=[0, 1, 0], aperture=np.pi / 3)

image = np.zeros((height, width, 3), dtype=np.uint8)

for y in range(height):
    for x in range(width):
        ray_direction = camera.generate_ray_direction(x, y, width, height)

        color = trace_ray(camera.position, ray_direction, objects, lights)

        image[y, x] = color

image = Image.fromarray(image)
image.save("output.png")
image.show()
